# Ensamble Stacking (Tugas 12)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Proses


*   Baca data *Diabetic Retinopathy Debrecen Data Set*
*   Buat data tes & data latih
*   Cari akurasi dari algoritma

### Persiapan Lingkungan


*   Impor *library* yang dibutuhkan

In [3]:
### Data Wrangling 
import pandas as pd
import numpy as np
from scipy.io import arff
from collections import OrderedDict

### Modelling 
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

### ensamble stacking
from sklearn.ensemble import StackingClassifier

### Remove unnecessary warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# load data set
data = arff.loadarff("/content/drive/MyDrive/datamining/tugas/messidor_features.arff")

In [6]:
df = pd.DataFrame(data[0])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Class
0,1.0,1.0,22.0,22.0,22.0,19.0,18.0,14.0,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1.0,b'0'
1,1.0,1.0,24.0,24.0,22.0,18.0,16.0,13.0,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0.0,b'0'
2,1.0,1.0,62.0,60.0,59.0,54.0,47.0,33.0,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0.0,b'1'
3,1.0,1.0,55.0,53.0,53.0,50.0,43.0,31.0,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0.0,b'0'
4,1.0,1.0,44.0,44.0,44.0,41.0,39.0,27.0,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0.0,b'1'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,1.0,1.0,34.0,34.0,34.0,33.0,31.0,24.0,6.071765,0.937472,0.031145,0.003115,0.000000,0.000000,0.000000,0.000000,0.537470,0.116795,0.0,b'0'
1147,1.0,1.0,49.0,49.0,49.0,49.0,45.0,37.0,63.197145,27.377668,8.067688,0.979548,0.001552,0.000000,0.000000,0.000000,0.516733,0.124190,0.0,b'0'
1148,1.0,0.0,49.0,48.0,48.0,45.0,43.0,33.0,30.461898,13.966980,1.763305,0.137858,0.011221,0.000000,0.000000,0.000000,0.560632,0.129843,0.0,b'0'
1149,1.0,1.0,39.0,36.0,29.0,23.0,13.0,7.0,40.525739,12.604947,4.740919,1.077570,0.563518,0.326860,0.239568,0.174584,0.485972,0.106690,1.0,b'1'


In [7]:
#check number of rows and columns in dataset
df.shape

(1151, 20)

### Preprocessing Data

In [8]:
col_names = []
for i in range(20):
    if i == 0:
        col_names.append("quality")
    if i == 1:
        col_names.append("prescreen")
    if i >= 2 and i <= 7:
        col_names.append("ma" + str(i))
    if i >= 8 and i <= 15:
        col_names.append("exudate" + str(i))
    if i == 16:
        col_names.append("euDist")
    if i == 17:
        col_names.append("diameter")
    if i == 18:
        col_names.append("amfm_class")
    if i == 19:
        col_names.append("label")

In [9]:
df.columns = col_names

In [10]:
df

,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,1.0,1.0,22.0,22.0,22.0,19.0,18.0,14.0,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1.0,b'0'
1,1.0,1.0,24.0,24.0,22.0,18.0,16.0,13.0,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0.0,b'0'
2,1.0,1.0,62.0,60.0,59.0,54.0,47.0,33.0,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0.0,b'1'
3,1.0,1.0,55.0,53.0,53.0,50.0,43.0,31.0,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0.0,b'0'
4,1.0,1.0,44.0,44.0,44.0,41.0,39.0,27.0,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0.0,b'1'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,1.0,1.0,34.0,34.0,34.0,33.0,31.0,24.0,6.071765,0.937472,0.031145,0.003115,0.000000,0.000000,0.000000,0.000000,0.537470,0.116795,0.0,b'0'
1147,1.0,1.0,49.0,49.0,49.0,49.0,45.0,37.0,63.197145,27.377668,8.067688,0.979548,0.001552,0.000000,0.000000,0.000000,0.516733,0.124190,0.0,b'0'
1148,1.0,0.0,49.0,48.0,48.0,45.0,43.0,33.0,30.461898,13.966980,1.763305,0.137858,0.011221,0.000000,0.000000,0.000000,0.560632,0.129843,0.0,b'0'
1149,1.0,1.0,39.0,36.0,29.0,23.0,13.0,7.0,40.525739,12.604947,4.740919,1.077570,0.563518,0.326860,0.239568,0.174584,0.485972,0.106690,1.0,b'1'


### Create Normalize and Unnormalized data


```
X for data test
y for data label

```

In [11]:
# percent_amount_of_test_data = / HUNDRED_PERCENT
percent_amount_of_test_data = 0.2

In [12]:
unnormalized_data = df.drop(columns = ["label"])

In [13]:
y = df["label"].values

In [14]:
scaler = MinMaxScaler()
scala = scaler.fit(unnormalized_data)

In [15]:
normalized_dataset = scaler.transform(unnormalized_data)

In [16]:
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

*    Create an unormalized data set 

In [17]:
# split unnormalized data
X_train, X_test, y_train, y_test = train_test_split(unnormalized_data, y, test_size = percent_amount_of_test_data, random_state=42, shuffle=False)

*    Create an normalized dataset

In [18]:
# split normalized data
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(normalized_dataset, y, test_size = percent_amount_of_test_data, random_state=42, shuffle=False)

### Count & Compare Accuracy from Every Model

#### Gaussian Naive bayes


##### Accuracy of Normalization Data

In [19]:
clf_norm_nb = GaussianNB()
clf_norm_nb.fit(X_train_norm, y_train_norm)

GaussianNB()

In [20]:
post_norm_nb = clf_norm_nb.predict_proba(X_test_norm)
probas_norm_nb = post_norm_nb[:,1]
probas_norm_nb = np.round(probas_norm_nb)

In [21]:
pred_norm_nb = probas_norm_nb
accuracy_norm_nb = accuracy_score(y_test_norm, pred_norm_nb)

In [22]:
print(f"Akurasi dengan data ternormalisasi {accuracy_norm_nb} model Gaussian Naive Bayes")

Akurasi dengan data ternormalisasi 0.6363636363636364 model Gaussian Naive Bayes


##### Accuracy of Unnormalization Data

In [23]:
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)

GaussianNB()

In [24]:
post_nb = clf_nb.predict_proba(X_test)
probas_nb = post_nb[:,1]
probas_nb = np.round(probas_nb)

In [25]:
pred_nb = probas_nb
accuracy_nb = accuracy_score(y_test, pred_nb)

In [26]:
print(f"Akurasi dengan data tidak ternormalisasi {accuracy_nb} model Gaussian Naive Bayes")

Akurasi dengan data tidak ternormalisasi 0.645021645021645 model Gaussian Naive Bayes


#### K - Nearest Neighbors


In [27]:
# amount of K
k_is_3 = 3
k_is_5 = 5
k_is_7 = 7
k_is_10 = 10

##### Accuracy of Normalization Data

###### K 3

In [28]:
clf_norm_knn_3 = KNeighborsClassifier(n_neighbors = k_is_3)
clf_norm_knn_3.fit(X_train_norm, y_train_norm)

KNeighborsClassifier(n_neighbors=3)

In [29]:
post_norm_knn_3 = clf_norm_knn_3.predict_proba(X_test_norm)
probas_norm_knn_3 = post_norm_knn_3[:,1]
probas_norm_knn_3 = np.round(probas_norm_knn_3)

In [30]:
pred_norm_knn_3 = probas_norm_knn_3
accuracy_norm_knn_3 = accuracy_score(y_test_norm, pred_norm_knn_3)

In [31]:
print(f"Akurasi dengan data ternormalisasi {accuracy_norm_knn_3} model KNN-3")

Akurasi dengan data ternormalisasi 0.6320346320346321 model KNN-3


###### K 5

In [32]:
clf_norm_knn_5 = KNeighborsClassifier(n_neighbors = k_is_5)
clf_norm_knn_5.fit(X_train_norm, y_train_norm)

KNeighborsClassifier()

In [33]:
post_norm_knn_5 = clf_norm_knn_5.predict_proba(X_test_norm)
probas_norm_knn_5 = post_norm_knn_5[:,1]
probas_norm_knn_5 = np.round(probas_norm_knn_5)

In [34]:
pred_norm_knn_5 = probas_norm_knn_5
accuracy_norm_knn_5 = accuracy_score(y_test_norm, pred_norm_knn_5)

In [35]:
print(f"Akurasi dengan data ternormalisasi {accuracy_norm_knn_5} model KNN-5")

Akurasi dengan data ternormalisasi 0.6103896103896104 model KNN-5


###### K 7

In [36]:
clf_norm_knn_7 = KNeighborsClassifier(n_neighbors = k_is_7)
clf_norm_knn_7.fit(X_train_norm, y_train_norm)

KNeighborsClassifier(n_neighbors=7)

In [37]:
post_norm_knn_7 = clf_norm_knn_7.predict_proba(X_test_norm)
probas_norm_knn_7 = post_norm_knn_7[:,1]
probas_norm_knn_7 = np.round(probas_norm_knn_7)

In [38]:
pred_norm_knn_7 = probas_norm_knn_7
accuracy_norm_knn_7 = accuracy_score(y_test_norm, pred_norm_knn_7)

In [39]:
print(f"Akurasi dengan data ternormalisasi {accuracy_norm_knn_7} model KNN-7")

Akurasi dengan data ternormalisasi 0.645021645021645 model KNN-7


###### K 10

In [40]:
clf_norm_knn_10 = KNeighborsClassifier(n_neighbors = k_is_10)
clf_norm_knn_10.fit(X_train_norm, y_train_norm)

KNeighborsClassifier(n_neighbors=10)

In [41]:
post_norm_knn_10 = clf_norm_knn_10.predict_proba(X_test_norm)
probas_norm_knn_10 = post_norm_knn_10[:,1]
probas_norm_knn_10 = np.round(probas_norm_knn_10)

In [42]:
pred_norm_knn_10 = probas_norm_knn_10
accuracy_norm_knn_10 = accuracy_score(y_test_norm, pred_norm_knn_10)

In [43]:
print(f"Akurasi dengan data ternormalisasi {accuracy_norm_knn_10} model KNN-10")

Akurasi dengan data ternormalisasi 0.683982683982684 model KNN-10


##### Accuracy of Unormalization Data

###### K 3

In [44]:
clf_knn_3 = KNeighborsClassifier(n_neighbors = k_is_3)
clf_knn_3.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [45]:
post_knn_3 = clf_knn_3.predict_proba(X_test)
probas_knn_3 = post_knn_3[:,1]
probas_knn_3 = np.round(probas_knn_3)

In [46]:
pred_knn_3 = probas_knn_3
accuracy_knn_3 = accuracy_score(y_test, pred_knn_3)

In [47]:
print(f"Akurasi dengan data tidak ternormalisasi {accuracy_knn_3} model KNN-3")

Akurasi dengan data tidak ternormalisasi 0.658008658008658 model KNN-3


###### K 5

In [48]:
clf_knn_5 = KNeighborsClassifier(n_neighbors = k_is_5)
clf_knn_5.fit(X_train, y_train)

KNeighborsClassifier()

In [49]:
post_knn_5 = clf_knn_5.predict_proba(X_test)
probas_knn_5 = post_knn_5[:,1]
probas_knn_5 = np.round(probas_knn_5)

In [50]:
pred_knn_5 = probas_knn_5
accuracy_knn_5 = accuracy_score(y_test, pred_knn_5)

In [51]:
print(f"Akurasi dengan data tidak ternormalisasi {accuracy_knn_5} model KNN-5")

Akurasi dengan data tidak ternormalisasi 0.658008658008658 model KNN-5


###### K 7

In [52]:
clf_knn_7 = KNeighborsClassifier(n_neighbors = k_is_7)
clf_knn_7.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [53]:
post_knn_7 = clf_knn_7.predict_proba(X_test)
probas_knn_7 = post_knn_7[:,1]
probas_knn_7 = np.round(probas_knn_7)

In [54]:
pred_knn_7 = probas_knn_7
accuracy_knn_7 = accuracy_score(y_test, pred_knn_7)

In [55]:
print(f"Akurasi dengan data tidak ternormalisasi {accuracy_knn_7} model KNN-7")

Akurasi dengan data tidak ternormalisasi 0.6666666666666666 model KNN-7


###### K 10

In [56]:
clf_knn_10 = KNeighborsClassifier(n_neighbors = k_is_10)
clf_knn_10.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [57]:
post_knn_10 = clf_knn_10.predict_proba(X_test)
probas_knn_10 = post_knn_10[:,1]
probas_knn_10 = np.round(probas_knn_10)

In [58]:
pred_knn_10 = probas_knn_10
accuracy_knn_10 = accuracy_score(y_test, pred_knn_10)

In [59]:
print(f"Akurasi dengan data tidak ternormalisasi {accuracy_knn_10} model KNN-10")

Akurasi dengan data tidak ternormalisasi 0.6623376623376623 model KNN-10


### Ensemble Stacking


> Teknik menggunakan banyak model untuk menghasilkan satu model prediksi terbaik 



Contoh **Stack Classifier**


*   *gnb* adalah Gaussian Naive Bayes
*   *knn-3* adalah KNN dengan K = 3
*   *knn-5* adalah KNN dengan K = 5
*   *knn-7* adalah KNN dengan K = 7
*   *knn-10* adalah KNN dengan K = 10


```
[
    ('gnb',clf_norm_nb), 
    ('knn-3',clf_norm_knn_3), 
    ('knn-5',clf_norm_knn_5),
    ('knn-7',clf_norm_knn_7), 
    ('knn-10',clf_norm_knn_10)
]
```



#### Ensemble Stacking dengan data yang sudah di normalisasi

In [60]:
estimators_norm = [('gnb',clf_norm_nb),('knn-5',clf_norm_knn_5),('knn-7',clf_norm_knn_7)]

In [61]:
final_estimator_dtc = DecisionTreeClassifier(random_state=42)
final_estimator_lr = LogisticRegression()
final_estimator_knn_7 = KNeighborsClassifier(n_neighbors = k_is_7)

##### Meta Classifier is Decision Tree Classifier

In [62]:
stack_model = StackingClassifier( estimators = estimators_norm,final_estimator = final_estimator_dtc)
score = cross_val_score(stack_model,X_train_norm,y_train_norm,cv = 5,scoring = 'accuracy')
print("The accuracy score of is:",score.mean())

The accuracy score of is: 0.5945652173913044


##### Meta Classifier is Logistic Regression

In [63]:
stack_model = StackingClassifier( estimators = estimators_norm,final_estimator = final_estimator_lr)
score = cross_val_score(stack_model,X_train_norm,y_train_norm,cv = 5,scoring = 'accuracy')
print("The accuracy score of is:",score.mean())

The accuracy score of is: 0.6543478260869565


##### Meta Classifier is KNN (K = 7)

In [64]:
stack_model = StackingClassifier( estimators = estimators_norm,final_estimator = final_estimator_knn_7)
score = cross_val_score(stack_model,X_train_norm,y_train_norm,cv = 5,scoring = 'accuracy')
print("The accuracy score of is:",score.mean())

The accuracy score of is: 0.6130434782608696


#### Ensemble Stacking dengan data yang **tidak** di normalisasi

In [65]:
estimators_unnorm = [('gnb',clf_nb),('knn-5',clf_knn_5),('knn-7',clf_knn_7)]

In [66]:
final_estimator_dtc = DecisionTreeClassifier(random_state=42)
final_estimator_lr = LogisticRegression()
final_estimator_knn_7 = KNeighborsClassifier(n_neighbors = k_is_7)

##### Meta Classifier is Decision Tree Classifier

In [67]:
stack_model = StackingClassifier( estimators = estimators_unnorm,final_estimator = final_estimator_dtc)
score = cross_val_score(stack_model,X_train, y_train,cv = 5,scoring = 'accuracy')
print("The accuracy score of is:",score.mean())

The accuracy score of is: 0.601086956521739


##### Meta Classifier is Logistic Regression

In [68]:
stack_model = StackingClassifier( estimators = estimators_unnorm,final_estimator = final_estimator_lr)
score = cross_val_score(stack_model,X_train, y_train,cv = 5,scoring = 'accuracy')
print("The accuracy score of is:",score.mean())

The accuracy score of is: 0.6619565217391304


##### Meta Classifier is KNN (K = 7)

In [71]:
stack_model = StackingClassifier( estimators = estimators_unnorm,final_estimator = final_estimator_knn_7)
score = cross_val_score(stack_model,X_train, y_train,cv = 5,scoring = 'accuracy')
print("The accuracy score of is:",score.mean())

The accuracy score of is: 0.6228260869565216
